In [1]:
from subprocess import check_output
import json, os

In [2]:
data = "/Users/heidihurst/Documents/UK/ucl-gis/dissertation/data"

In [3]:
gml_head = '<?xml version="1.0" encoding="utf-8" ?><ogr:FeatureCollection xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="" xmlns:ogr="http://ogr.maptools.org/" xmlns:gml="http://www.opengis.net/gml">'                                                                                          

gml_foot = "</ogr:FeatureCollection>"

In [4]:
outfiledirectory = data
outfilename = "photo_bounds"

In [5]:
out_gml = gml_head

In [6]:
def parse_coords(coord_list):
    out_coords = ""
    for coords in coord_list:
        out_coords += str(coords[0]) + "," + str(coords[1]) + " "
    out_coords += str(coord_list[0][0]) + "," + str(coord_list[0][1])
    
    return out_coords

In [7]:
# initialize counter
i = 0
gml_middle = ""
for filename in os.listdir(data):
    if filename.endswith(".tif"):
        # get gdal data
        tiff_info = json.loads(check_output(["gdalinfo","-json",
                                             "/Users/heidihurst/Documents/UK/ucl-gis/dissertation/data/"+filename]))
        # get just file name
        image_name = os.path.splitext(filename)[0]
        
        # add field info
        gml_middle += " <gml:featureMember> <ogr:" + outfilename + " fid=" + '"' + outfilename + '.' + str(i) + '/">'
        gml_middle += '<ogr:geometryProperty> <gml:Polygon srsName="EPSG:4326"> <gml:outerBoundaryIs> <gml:LinearRing> <gml:coordinates>'
        
        # add coordinate info
        gml_middle += parse_coords(tiff_info['wgs84Extent']['coordinates'][0])
        
        # close polygon
        gml_middle += "</gml:coordinates> </gml:LinearRing> </gml:outerBoundaryIs> </gml:Polygon> </ogr:geometryProperty> "
        
        # add ID field
        gml_middle += "<ogr:id>" + str(i) + "</ogr:id>"
        
        # add name field equal to original tif name and close tag
        gml_middle += "<ogr:name> " + image_name +" </ogr:name>"
        
        # add close tag
        gml_middle += "</ogr:" + outfilename + "> </gml:featureMember>"

In [8]:
out_gml = gml_head + gml_middle + gml_foot

In [9]:
out_gml_path = os.path.join(outfiledirectory, outfilename+'.gml')
with open(out_gml_path, "w") as f:
    f.write(out_gml)

In [11]:
tiff_info['size']

[1688, 1356]

In [13]:
"banana" in ['banana foster']

False

In [14]:
tiff_info['size']


[1688, 1356]

In [15]:
json.loads(check_output(["gdalinfo","-json","/Users/heidihurst/Documents/UK/ucl-gis/dissertation/data/104.tif"]))['size']

[3377, 2713]

In [17]:
os.path.basename("/Users/heidihurst/Documents/UK/ucl-gis/dissertation/data/104.tif")

'104.tif'

In [25]:
import math
from subprocess import call

In [40]:
def downsample_image(image_path, downsample_factor=2.0, resample_method='lanczos', output_location=os.path.join(os.getcwd(),'downsampled')):
    # get image size from gdalinfo
    size = json.loads(check_output(["gdalinfo","-json", image_path]))['size'][0]
    size = math.ceil(size/float(downsample_factor))
    # call gdalwarp to create new image
    image_name = os.path.basename(image_path)
    print(output_location)
    call(["gdalwarp -ts " + str(size) + " 0 -r " + resample_method + " " + image_path + " " + os.path.join(output_location, image_name)], shell=True)

In [41]:
downsample_image("104.tif")

/Users/heidihurst/Documents/UK/ucl-gis/dissertation/dissertation/downsampled


In [31]:
os.path.join("/downsampled","104.tif")

'/downsampled/104.tif'

In [33]:
os.getcwd()

'/Users/heidihurst/Documents/UK/ucl-gis/dissertation/dissertation'

In [5]:
gvers = check_output(['gdalinfo', '--version'])

In [6]:
# get which version of gdal
# get everything after "GDAL "
"2." in gvers

True

In [7]:
"1." in gvers

False

In [10]:
" 2." in "1.2.1"

False

In [11]:
def parse_coords(image_path):
    """
    Correctly formats coordinates list for use in gml string, including
    ensuring that it is closed (e.g. first coordinate is the same as last coordinate)

    Args:
        coord_list: a nested list containing coordinates in the form [[lat,lon],[lat,lon]]
                    must have four entries

    Output:
        returns formmated string
    """

    # determine version
    gdal_version = check_output(['gdalinfo','--version'])

    # if 2.0 or higher, parse json
    if " 2." in gdal_version:
        # get gdal data
        tiff_info = json.loads(check_output(["gdalinfo","-json", image_path]))
        coord_list = tiff_info['wgs84Extent']['coordinates'][0]
        out_coords = ""
        for coords in coord_list:
            out_coords += str(coords[0]) + "," + str(coords[1]) + " "
        out_coords += str(coord_list[0][0]) + "," + str(coord_list[0][1])
    # if under 2.0, no json to parse
    elif " 1." in gdal_version:
        # get gdal data as coord
        

In [64]:
def get_coords(image_path):
    tiff_info = check_output(['gdalinfo '+image_path], shell=True)
    
    # get coordinate section
    tiff_info = tiff_info[:tiff_info.index('Center')]
    
    coords = []
    # get first coordinate set, then cut string
    for location in ["Upper Left", "Lower Left", "Upper Right", "Lower Right"]:
        info = tiff_info[tiff_info.index(location):]
        info = info[info.index('(')+1:info.index(')')]
        # split coordinates and make floats
        coord = [float(x) for x in info.split(",")]
        coords.append(coord)
    
    return coords

In [28]:
path = '/Users/heidihurst/Documents/UK/ucl-gis/dissertation/data/100.tif'

In [66]:
outcoords=get_coords(path)

In [31]:
info = check_output(['gdalinfo '+path], shell=True)

In [43]:
info[info.index('Upper Left'):info.index('Center')]
#info[:info.index('Center')]

'Upper Left  (  10.2520059,  36.8719731) ( 10d15\' 7.22"E, 36d52\'19.10"N)\nLower Left  (  10.2520059,  36.8628427) ( 10d15\' 7.22"E, 36d51\'46.23"N)\nUpper Right (  10.2633735,  36.8719731) ( 10d15\'48.14"E, 36d52\'19.10"N)\nLower Right (  10.2633735,  36.8628427) ( 10d15\'48.14"E, 36d51\'46.23"N)\n'

In [49]:
ul = info[info.index('(')+1:info.index(')')]

In [68]:
outcoords[1]

[10.2520059, 36.8628427]

In [53]:
coords = []

In [54]:
coords.append([])

In [55]:
coords

[[]]

In [60]:
coord = [float(x) for x in ul.split(',')]

In [61]:
coord

[10.25200594232203, 36.871973134459566]